In [1]:
import rich

from mdmodels import DataModel

In [2]:
# Parse the markdown file
dm = DataModel.from_markdown('model.md')

In [3]:
# Use the data model
project = dm.ChemicalProject(title='My Project')

mol1 = dm.Molecule(name='Molecule 1', smiles='CCO')
mol2 = dm.Molecule(name='Molecule 2', smiles='CCN')

project.molecules.extend([mol1, mol2])

In [4]:
# To JSON
rich.print(project.model_dump_json(indent=2))

{
  "title": "My Project",
  "molecules": [
    {
      "id": null,
      "name": "Molecule 1",
      "formula": null
    },
    {
      "id": null,
      "name": "Molecule 2",
      "formula": null
    }
  ],
  "reactions": [],
  "experiments": []
}

In [5]:
# To XML
rich.print(project.xml())

<?xml version="1.0" ?>
<ChemicalProject>
  <title>My Project</title>
  <molecules>
    <id/>
    <name>Molecule 1</name>
    <formula/>
  </molecules>
  <molecules>
    <id/>
    <name>Molecule 2</name>
    <formula/>
  </molecules>
</ChemicalProject>